In [ ]:
# faceMask hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 12/07/20

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data = hddm.utils.flip_errors(data)

# check dataset
data.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (disgusted) and facial expression (angry)
da_data = data.loc[(data['emotionRating'] == 'disgusted') & (data['expression'] == 'angry')]
da_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
# none for da

In [ ]:
# define model
m_da = hddm.HDDMRegressor(da_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_da.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_da.print_stats()
m_da.plot_posteriors()

In [ ]:
# save model stats
da_stats = m_da.gen_stats()
print(da_stats)
da_stats.to_csv('da_5000/faceMask_hddm_drift_maskFalsePositives_da_5000.csv', index=True)

# save model posterior plots
m_da.plot_posteriors(path='da_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
da_v_none, da_v_lower, da_v_upper = m_da.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([da_v_none, da_v_lower, da_v_upper])
plt.savefig('da_5000/faceMask_hddm_drift_maskFalsePositives_da_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(da_v_none < 0) = ", (da_v_none.trace() < 0).mean())
print("P(da_v_lower < 0) = ", (da_v_lower.trace() < 0).mean())
print("P(da_v_upper > 0) = ", (da_v_upper.trace() > 0).mean())

# estimate probability that lower and upper mask coefficient posteriors differ
print("P(da_v_lower < da_v_upper) = ", (da_v_lower.trace() < da_v_upper.trace()).mean())

In [ ]:
# subset dataset by emotion rating type (disgusted) and facial expression (fearful)
df_data = data.loc[(data['emotionRating'] == 'disgusted') & (data['expression'] == 'fearful')]
df_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error) 
df_data = df_data[(df_data.subj_idx != 'jkgqk1co77rw24c') & (df_data.subj_idx != 'me8doxrmo9vj9dx') & (df_data.subj_idx != 'xs5439nm2v85thb')]

In [ ]:
# define model
m_df = hddm.HDDMRegressor(df_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_df.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_df.print_stats()
m_df.plot_posteriors()

In [ ]:
# save model stats
df_stats = m_df.gen_stats()
print(df_stats)
df_stats.to_csv('df_5000/faceMask_hddm_drift_maskFalsePositives_df_5000.csv', index=True)

# save model posterior plots
m_df.plot_posteriors(path='df_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
df_v_none, df_v_lower, df_v_upper = m_df.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([df_v_none, df_v_lower, df_v_upper])
plt.savefig('df_5000/faceMask_hddm_drift_maskFalsePositives_df_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(df_v_none < 0) = ", (df_v_none.trace() < 0).mean())
print("P(df_v_lower < 0) = ", (df_v_lower.trace() < 0).mean())
print("P(df_v_upper > 0) = ", (df_v_upper.trace() > 0).mean())

# estimate probability that lower and upper mask coefficient posteriors differ
print("P(df_v_lower < df_v_upper) = ", (df_v_lower.trace() < df_v_upper.trace()).mean())